In [ ]:
# ==============================================================================
# Step 1: 필요한 도구들 불러오기 (Setup)
# ==============================================================================
import os
import deepl
import pandas as pd
from dotenv import load_dotenv, find_dotenv # ★★★ FIX 1: 'find_dotenv' 탐정 추가 ★★★
from tqdm.notebook import tqdm
from google.cloud import translate_v2 as translate
import requests
import time

# ==============================================================================
# Step 2: 실험 조건 설정하기 (Configuration)
# ==============================================================================
# --- API 키 설정 ---
# ★★★ FIX 1 (적용): find_dotenv()를 사용해 어디서든.env 파일을 찾도록 설정 ★★★
load_dotenv(find_dotenv())
DEEPL_API_KEY = os.getenv('DEEPL_API_KEY')

# --- 실험 변수 설정 ---
engines_to_test = ['DeepL', 'Google', 'LibreTranslate']
pivot_language = 'DE'
iterations = 1
num_samples_to_test = 10
source_data_path = 'Esperanto-Dataset/Dataset/News_AI.csv'

# ==============================================================================
# Step 3: 각 번역기별 역번역 함수 만들기 (Helper Functions)
# ==============================================================================
# --- DeepL 번역 함수 ---
def back_translate_deepl(text, pivot_lang, num_iterations):
    if not DEEPL_API_KEY:
        return "Error: DEEPL_API_KEY not found. Check.env file."
    translator = deepl.Translator(DEEPL_API_KEY)
    current_text = text
    try:
        for _ in range(num_iterations):
            to_pivot = translator.translate_text(current_text, target_lang=pivot_lang).text
            time.sleep(1)
            to_english = translator.translate_text(to_pivot, target_lang="EN-US").text
            current_text = to_english
        return current_text
    except Exception as e:
        return f"Error: {e}"

# --- Google Translate 번역 함수 (안정성 개선) ---
def back_translate_google(text, pivot_lang, num_iterations):
    client = translate.Client()
    current_text = text
    try:
        for _ in range(num_iterations):
            # ★★★ FIX 2: Google 번역 결과(딕셔너리)에서 'translatedText' 키로 텍스트를 정확히 추출 ★★★
            to_pivot_result = client.translate(current_text, target_language=pivot_lang)
            translated_pivot_text = to_pivot_result
            
            time.sleep(1)
            
            to_english_result = client.translate(translated_pivot_text, target_language='en')
            current_text = to_english_result
        return current_text
    except Exception as e:
        return f"Error: {e}"

# --- LibreTranslate 번역 함수 (안정성 개선) ---
def back_translate_libre(text, pivot_lang, num_iterations):
    url = "https://libretranslate.de/translate"
    current_text = text
    try:
        for _ in range(num_iterations):
            params_to_pivot = {"q": current_text, "source": "en", "target": pivot_lang.lower()}
            response_to_pivot = requests.post(url, json=params_to_pivot)
            # ★★★ FIX 3: LibreTranslate 결과(json)에서 'translatedText' 키로 텍스트를 정확히 추출 ★★★
            translated_pivot_text = response_to_pivot.json()
            
            time.sleep(1)

            params_to_english = {"q": translated_pivot_text, "source": pivot_lang.lower(), "target": "en"}
            response_to_english = requests.post(url, json=params_to_english)
            current_text = response_to_english.json()
        return current_text
    except Exception as e:
        return f"Error: {e}"

# ==============================================================================
# Step 4: 실제 실험 실행하기 (Main Experiment Loop) - 변경 없음
# ==============================================================================
print(f"Loading original data from {source_data_path}...")
original_df = pd.read_csv(source_data_path)
original_texts = original_df['AI'].head(num_samples_to_test).tolist()
print(f"Loaded {len(original_texts)} text samples for the test run.")

all_results = []

for engine_name in engines_to_test:
    print(f"\n===== Starting experiment for engine: {engine_name} =====")
    
    for original_text in tqdm(original_texts, desc=f"Translating with {engine_name}"):
        translated_text = None
        if engine_name == 'DeepL':
            translated_text = back_translate_deepl(original_text, pivot_lang=pivot_language, num_iterations=iterations)
        elif engine_name == 'Google':
            translated_text = back_translate_google(original_text, pivot_lang=pivot_language, num_iterations=iterations)
        elif engine_name == 'LibreTranslate':
            translated_text = back_translate_libre(original_text, pivot_lang=pivot_language, num_iterations=iterations)
        
        all_results.append({
            'original_text': original_text,
            'translated_text': translated_text,
            'pivot_language': pivot_language,
            'iterations': iterations,
            'engine': engine_name
        })

print("\n===== All translation processes completed. =====")

results_df = pd.DataFrame(all_results)
output_filename = f"all_engines_results_{pivot_language}_{iterations}_iter.csv"
results_df.to_csv(output_filename, index=False)

print(f"\nSuccessfully saved all results to '{output_filename}'")
print("\n--- First 10 rows of the combined result ---")
print(results_df.head(10))

SyntaxError: invalid syntax (2263876767.py, line 93)